# AmgX_Semi_Implicite

### Description 
Several GPU solvers from AmgX library used by TRUST are tested: Gradient Conjugate (GC) and FGMRES (Flexible GMRES) with GS (Gauss-Seidel) or SSOR preconditioners. Intriguely, convergence is much faster with GC+GC on GPU than GC+SSOR on CPU. Should implement GS for PETSc to check cause GC without preconditioner give same results on CPU and GPU.

In [ ]:
from trustutils import run  
run.TRUST_parameters()
run.introduction('Pierre LEDAC')
cases=[("amgx_gc","AmgX CG"),("amgx_gc_gs","AmgX CG/GS"),("amgx_fgmres_gs","AmgX FGMRES/GS"),("petsc_gc","PETSc CG"),("petsc_gc_ssor","PETSc CG/SSOR")]
mesh="10x10x10"
for case,label in cases:
    run.addCase(mesh ,"%s.data" % case, 2)
run.printCases()

In [ ]:
run.runCases()

# Solver convergence with different solvers on mesh 40x10x10x10

In [ ]:
from trustutils import plot
    
a = plot.Graph("Relative residual ||Ax(it)-b||/||Ax(0)-b|| during the fist time step:","",1,1,[10,5])

for case,label in cases:
    cols = plot.loadText(mesh+"/%s.res" % case)
    a.add(cols[0],cols[1],label="%s" % label)

a.label("Iteration","Residual")
a.subplot.set_yscale('log')

# Time convergence with different solvers on mesh 40x10x10x10

In [ ]:
a = plot.Graph("Relative residual ||dI(n)/dt||/||dI(0)/dt|| up to convergence:","",1,1,[10,5])

for case,label in cases:
    cols = plot.loadText(mesh+"/%s.dt_ev" % case)
    a.add(cols[0],cols[3],label="%s" % label)

a.label("Time [s]","Residual")
a.subplot.set_yscale('log')

***The time convergence are identical for all GPU solvers compared to PETSc CPU versions.***

# Friction drag with different solvers on mesh 40x10x10x10

In [ ]:
a = plot.Graph("Fx(Wall) up to convergence:","",1,1,[10,5])
for case,label in cases:
    cols = plot.loadText(mesh+"/%s_pb_Contrainte_visqueuse.out" % case)
    a.add(cols[0],cols[4],label="%s" % label)
a.label("Time [s]","Drag [N]")

# RAM evolution for different solvers on mesh 40x10x10x10

In [ ]:
a = plot.Graph("RAM used during calculation:","",1,1,[10,5])
for case,label in cases:
    cols = plot.loadText(mesh+"/%s.ram" % case)
    a.add(cols[0],cols[1],label="%s" % label)
a.label("Time [s]","RAM [MB]")

## Time needed to copy and solve the linear system on GPU (during the last time step)

In [ ]:
headers=["Copy[s]","Solve[s]","Total[s]"]
a = plot.Table(headers)
for case,label in cases:
    a.load(mesh+"/%s.cpu" % case,"%s" % label)

a.print()

# Conclusions

Solvers on GPU are validated and GC (or FGMRES) and GS are the optimal choice to use for a diffusive-implicited scheme. Warning: performance of GPU solvers indicated here are not relevant on this small test case and/or on a entry-level NVidia card (with minimal double precision support).

### Computer performance:

In [ ]:
run.tablePerf()